<a href="https://colab.research.google.com/github/rakesh4real/ocr.pytorch/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install tree
!git clone https://github.com/courao/ocr.pytorch.git

Cloning into 'ocr.pytorch'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 285 (delta 30), reused 70 (delta 24), pack-reused 207
Receiving objects: 100% (285/285), 2.23 MiB | 2.14 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [3]:
%cd ocr.pytorch

/content/ocr.pytorch


In [4]:
!tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (46.3 kB/s)
Selecting previously unselected package tree.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
.
├── checkpoints
├── demo.py
├── detect
│   ├── config.py
│   ├── ctpn_model.py
│   ├── ctpn_predict.py
│   ├── ctpn_utils.py
│   └── __pycache__
│       ├── config.cpython-37.pyc
│       ├── ctpn_model.cpython-37.pyc
│       ├── ctpn_predict.cpython-37.pyc
│       └── ctpn_uti

In [7]:
!cp '/content/drive/My Drive/ocr/CRNN-1010.pth' ./checkpoints/
!cp '/content/drive/My Drive/ocr/CTPN.pth' ./checkpoints/

In [9]:
!ls checkpoints

CRNN-1010.pth  CTPN.pth


In [18]:
!python test_one.py testme.png

{0: [array([ 20.94909918,  19.64116265, 505.        ,  18.79504979,
        21.        ,  47.55773588, 505.05090082,  46.71162301,
         0.93806547]), 'There isanenvironmental varable.'], 1: [array([5.00000000e+00, 6.22860174e+01, 1.03300733e+03, 6.25257055e+01,
       4.99267409e+00, 9.30951073e+01, 1.03300000e+03, 9.33347954e+01,
       9.16586876e-01]), 'TCMALLOCLARGEALLOCREPORT THRESHOLD. which bv defaut'], 2: [array([5.00000000e+00, 1.07393454e+02, 1.03302097e+03, 1.08063612e+02,
       4.97903339e+00, 1.38930477e+02, 1.03300000e+03, 1.39600635e+02,
       9.28762734e-01]), 'is Set to1073741824bytes.-1GBSofthere isanaocation that is == or'], 3: [array([5.00000000e+00, 1.52620604e+02, 1.01702862e+03, 1.53491540e+02,
       4.97138080e+00, 1.85218941e+02, 1.01700000e+03, 1.86089877e+02,
       9.28208947e-01]), 'exceeds1GBofmemom.thismessage wilbe displayed in he ndsd.log.'], 4: [array([5.00000000e+00, 2.33856625e+02, 1.01700082e+03, 2.33884731e+02,
       4.99917934e+00, 2.62822